# Feature performance comparison

The purpose of this notebook is to compare the classification performance of the individual features, and their combination, for A. Thaliana

# Imports

In [1]:
import os
import sys
from IPython.display import display

sys.path.append('../src')
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SelectKBest, RFE, VarianceThreshold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone
from scipy.stats import shapiro
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator,TransformerMixin

from yellowbrick.features import ParallelCoordinates
from yellowbrick.features import Rank1D, Rank2D

import pandas as pd
import numpy as np
import seaborn as sns

from dataset.transporter_dataset import create_dataset
from dataset.cluster_fasta import cd_hit
from features.labels import fasta_to_labels
from features.compositions import calculate_composition_feature
from features.pssm import calculate_pssm_feature
from features.coexp import calculate_coexp_feature
from models.eval import nested_crossval
from visualization.feature_plots import create_plot

# Globals

In [2]:
N_THREADS = 16
IDENTITY_THRESHOLD=70

LOG_FILE = "../logs/meta_amino_sugar.log"
N_THREADS = 16
ORGANISM = "meta"

# Dataset

In [3]:
# Delete previous log
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w"):
        pass
# e coli, a thaliana, human
create_dataset(
    keywords_substrate_filter=["Amino-acid transport", "Sugar transport"],
    keywords_component_filter=["Transmembrane"],
    keywords_transport_filter=["Transport"],
    input_file="../data/raw/swissprot/uniprot-reviewed_yes.tab.gz",
    multi_substrate="integrate",
    verbose=True,
    outliers=["P76773", "Q47706", "P64550", "P02943", "P75733", "P69856"]
    + ["O81775", "Q9SW07", "Q9FHH5", "Q8S8A0", "Q3E965", "Q3EAV6", "Q3E8L0"]
    + ["Q9HBR0", "Q07837"],
    tax_ids_filter=[3702, 9606, 83333, 559292],
    output_tsv=f"../data/datasets/{ORGANISM}_amino_sugar.tsv",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_log=LOG_FILE,
)


,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
P0CD99,Sugar transport,Cell membrane;Membrane;Transmembrane,Transport,MPH2 YDL247W,Alpha-glucosides permease MPH2 (Maltose transp...,NaN,559292,MKNLSFLINRRKENTSDSNVYPGKAKSHEPSWIEMDDQTKKDGLDI...
P69801,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,manY pel ptsP b1818 JW1807,PTS system mannose-specific EIIC component (EI...,4.A.6.1.1,83333,MEITTLQIVLVFIVACIAGMGSILDEFQFHRPLIACTLVGIVLGDM...
Q9SFG0,Sugar transport,Membrane;Transmembrane,Symport;Transport,STP6 At3g05960 F2O10.8,Sugar transport protein 6 (Hexose transporter 6),2.A.1.1.56,3702,MAVVVSNANAPAFEAKMTVYVFICVMIAAVGGLIFGYDIGISGGVS...
Q9BWM7,Amino-acid transport,Membrane;Mitochondrion;Transmembrane,Transport,SFXN3,Sideroflexin-3,NaN,9606,MGELPLDINIQEPRWDQSTFLGRARHFFTVTDPRNLLLSGAQLEAS...
Q9ZVK6,Sugar transport,Cell membrane;Membrane;Transmembrane,Symport;Transport,SUC8 At2g14670 T6B13.9,Sucrose transport protein SUC8 (Sucrose permea...,NaN,3702,MSDLQAKNDVVAVDRQSSSSLADLDGPSPLRKMISVASIAAGIQFG...
...,...,...,...,...,...,...,...,...
Q04162,Sugar transport,Membrane;Transmembrane,Transport,YDR387C,Probable metabolite transport protein YDR387C,2.A.1.1.101,559292,MSTDESEDVYSDLYSIISQVTSNTANDIEQLPYALTFKTSLIFVGA...
P33361,Amino-acid transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,yehY b2130 JW2118,Glycine betaine uptake system permease protein...,3.A.1.12.15,83333,MTYFRINPVLALLLLLTAIAAALPFISYAPNRLVSGEGRHLWQLWP...
P39328,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,ytfT b4230 JW5753,Galactofuranose transporter permease protein YtfT,3.A.1.2.25,83333,MMPQSLPDTTTPKRRFRWPTGMPQLVALLLVLLVDSLVAPHFWQVV...


## Clustering

In [4]:
cd_hit(
    executable_location="cd-hit",
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    log_file=LOG_FILE,
    identity_threshold=IDENTITY_THRESHOLD,
    n_threads=N_THREADS,
    memory=4096,
    verbose=True,
)

## Annotations

In [5]:
df_annotations = pd.read_table(f"../data/datasets/{ORGANISM}_amino_sugar.tsv", index_col=0)
df_annotations.head()

,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
P0CD99,Sugar transport,Cell membrane;Membrane;Transmembrane,Transport,MPH2 YDL247W,Alpha-glucosides permease MPH2 (Maltose transp...,NaN,559292,MKNLSFLINRRKENTSDSNVYPGKAKSHEPSWIEMDDQTKKDGLDI...
P69801,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,manY pel ptsP b1818 JW1807,PTS system mannose-specific EIIC component (EI...,4.A.6.1.1,83333,MEITTLQIVLVFIVACIAGMGSILDEFQFHRPLIACTLVGIVLGDM...
Q9SFG0,Sugar transport,Membrane;Transmembrane,Symport;Transport,STP6 At3g05960 F2O10.8,Sugar transport protein 6 (Hexose transporter 6),2.A.1.1.56,3702,MAVVVSNANAPAFEAKMTVYVFICVMIAAVGGLIFGYDIGISGGVS...
Q9BWM7,Amino-acid transport,Membrane;Mitochondrion;Transmembrane,Transport,SFXN3,Sideroflexin-3,NaN,9606,MGELPLDINIQEPRWDQSTFLGRARHFFTVTDPRNLLLSGAQLEAS...
Q9ZVK6,Sugar transport,Cell membrane;Membrane;Transmembrane,Symport;Transport,SUC8 At2g14670 T6B13.9,Sucrose transport protein SUC8 (Sucrose permea...,NaN,3702,MSDLQAKNDVVAVDRQSSSSLADLDGPSPLRKMISVASIAAGIQFG...


# Feature generation

## Labels

In [6]:
fasta_to_labels(
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
)
pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
    index_col=0,
).labels.value_counts()

Sugar transport         181
Amino-acid transport    166
Name: labels, dtype: int64

## PSSM

In [7]:
for uniref_cluster_threshold in [50, 90]:
    for psiblast_iterations in [1, 3]:
        calculate_pssm_feature(
            input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
            output_tsv="../data/features/{}_amino_sugar_cluster{}_pssm_ur{}_{}it.tsv".format(
                ORGANISM,
                IDENTITY_THRESHOLD,
                uniref_cluster_threshold,
                psiblast_iterations,
            ),
            tmp_folder="../data/intermediate/blast/pssm_uniref{}_{}it".format(
                uniref_cluster_threshold, psiblast_iterations
            ),
            blast_db="../data/raw/uniref/uniref{}/uniref{}.fasta".format(
                uniref_cluster_threshold, uniref_cluster_threshold
            ),
            iterations=psiblast_iterations,
            psiblast_executable="psiblast",
            psiblast_threads=N_THREADS,
            verbose=False,
        )


## Reading dataframes

In [8]:
df_labels = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
    index_col=0,
)
df_pssm_50_1it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur50_1it.tsv",
    index_col=0,
)
df_pssm_50_3it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur50_3it.tsv",
    index_col=0,
)
df_pssm_90_1it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur90_1it.tsv",
    index_col=0,
)
df_pssm_90_3it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur90_3it.tsv",
    index_col=0,
)

## Combining dataframes

In [9]:
df_pssm_all = pd.concat(
    [
        df_pssm_50_1it.rename(columns=lambda c: c + "_50_1"),
        df_pssm_50_3it.rename(columns=lambda c: c + "_50_3"),
        df_pssm_90_1it.rename(columns=lambda c: c + "_90_1"),
        df_pssm_90_3it.rename(columns=lambda c: c + "_90_3"),
    ],
    axis=1,
)


## Custom Transformer to try all parameters

In [10]:
class PSSMSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names, uniref_threshold="all", iterations="all"):
        self.feature_names = feature_names
        self.uniref_threshold = uniref_threshold
        self.iterations = iterations

    def fit(self, X, y=None):
        if self.uniref_threshold in {50, 90}:
            has_uniref = (
                np.char.find(self.feature_names, str(self.uniref_threshold)) >= 0
            )
        elif self.uniref_threshold == "all":
            has_uniref = np.array([True] * len(self.feature_names))
        else:
            raise ValueError(f"Incorrect uniref threshold {self.uniref_threshold}")

        if self.iterations in {1, 3}:
            has_iterations = np.char.find(self.feature_names, str(self.iterations)) >= 0
        elif self.iterations == "all":
            has_iterations = np.array([True] * len(self.feature_names))
        else:
            raise ValueError(f"Incorrect iteration count: {self.iterations}")
        self.mask = np.bitwise_and(has_uniref, has_iterations)
        return self

    def transform(self, X, y=None):
        X = np.array(X)
        X = X[:, self.mask]
        return X


# Functions

In [11]:
def get_feature_stats(df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"]):
    df_stats = pd.concat(
        {
            "corr": df_features.corrwith(
                df_labels_.labels.transform(lambda x: 1.0 if x == labels[1] else 0.0)
            ),
            "mean": df_features.mean(),
            "std": df_features.std(),
        },
        axis=1,
    )

    df_stats["sum"] = df_stats.sum(axis=1)
    df_stats["corr_abs"] = df_stats["corr"].abs()

    df_stats["mean0"] = df_features.loc[df_labels_[df_labels_.labels == labels[0]].index].mean()
    df_stats["mean1"] = df_features.loc[df_labels_[df_labels_.labels == labels[1]].index].mean()

    df_stats["median0"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[0]].index
    ].median()
    df_stats["median1"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[1]].index
    ].median()

    df_stats["mediandiff"] = (df_stats["median0"] - df_stats["median1"]).abs()
    df_stats = df_stats.sort_values("mediandiff", ascending=False)
    return df_stats

In [12]:
def get_independent_test_set(
    df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"], test_size=0.2
):
    X = df_features.to_numpy()
    y = np.where(df_labels_.labels == labels[1], 1, 0)
    feature_names = df_features.columns.to_numpy(dtype=str)
    sample_names = df_features.index.to_numpy(dtype=str)
    (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
    ) = train_test_split(
        X, y, sample_names, stratify=y, random_state=42, shuffle=True, test_size=test_size
    )
    return (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
        feature_names,
    )


In [13]:
def print_validation_results(y_true_, y_pred_, labels = ["Amino", "Sugar"]):
    report_dict = classification_report(y_true=y_true_, y_pred=y_pred_, output_dict=True)
    report_dict = {
        labels[0]: report_dict['0'],
        labels[1]: report_dict['1'],
        "Macro": report_dict["macro avg"],
        "Weighted": report_dict["weighted avg"]
    }
    report_df = pd.DataFrame.from_dict(report_dict)
    confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true_, y_pred_),
        columns=labels,
        index=labels,
    )
    return report_df, confusion_matrix_df

# Individual Features

## PSSM

### Stats, Plots

In [14]:
df_stats = get_feature_stats(df_pssm_50_1it, df_labels)


df_stats["shapiro_p"] = df_pssm_50_1it.apply(lambda col: shapiro(col)[1], axis=0).round(4)
df_stats["shapiro"] = df_pssm_50_1it.apply(lambda col: shapiro(col)[0], axis=0)

display(df_stats[df_stats.shapiro < 0.9])

df_stats.sort_values("std")

,corr,mean,std,sum,corr_abs,mean0,mean1,median0,median1,mediandiff,shapiro_p,shapiro
IL,-0.196008,0.918930,0.085069,0.807991,0.196008,0.936316,0.902985,0.975968,0.91023,0.065739,0.0,0.861988
QG,0.033406,0.932815,0.078602,1.044823,0.033406,0.930077,0.935326,0.944099,1.00000,0.055901,0.0,0.826964
ID,0.134605,0.007264,0.024399,0.166268,0.134605,0.003840,0.010405,0.000000,0.00000,0.000000,0.0,0.336061


,corr,mean,std,sum,corr_abs,mean0,mean1,median0,median1,mediandiff,shapiro_p,shapiro
ID,0.134605,0.007264,0.024399,0.166268,0.134605,0.003840,0.010405,0.000000,0.000000,0.000000,0.0000,0.336061
IE,0.257133,0.080826,0.031987,0.369945,0.257133,0.072250,0.088691,0.070531,0.081897,0.011365,0.0000,0.922364
IK,0.169175,0.124821,0.034073,0.328070,0.169175,0.118811,0.130334,0.116456,0.125628,0.009172,0.0000,0.965957
IN,0.196753,0.115180,0.038614,0.350546,0.196753,0.107258,0.122445,0.105598,0.115152,0.009554,0.0001,0.979976
IR,0.044217,0.146820,0.039746,0.230784,0.044217,0.144988,0.148501,0.143365,0.143813,0.000447,0.0001,0.979518
...,...,...,...,...,...,...,...,...,...,...,...,...
GH,0.039511,0.263300,0.106538,0.409349,0.039511,0.258911,0.267325,0.251109,0.274419,0.023309,0.0100,0.988956
GE,0.065808,0.226507,0.108146,0.400461,0.065808,0.219087,0.233313,0.214742,0.236220,0.021479,0.0066,0.988230
QL,-0.130242,0.256370,0.108211,0.234339,0.130242,0.271066,0.242893,0.279594,0.245399,0.034196,0.0016,0.985691
QW,-0.120714,0.299058,0.110157,0.288502,0.120714,0.312924,0.286342,0.314875,0.291358,0.023517,0.0357,0.991158


### Independent test set

In [15]:
(
    X_train,
    X_test,
    y_train,
    y_test,
    sample_names_train,
    sample_names_test,
    feature_names,
) = get_independent_test_set(df_pssm_all, df_labels, test_size=0.2)

### Model selection

SVC (with default RBF kernel) looks the most promising.

In [16]:
for estimator in [
    LinearSVC(max_iter=1e6, class_weight="balanced", random_state=0),
    SVC(class_weight="balanced"),
    RandomForestClassifier(class_weight="balanced", random_state=0),
    LinearSVC(max_iter=1e6, random_state=0),
    SVC(),
    RandomForestClassifier(random_state=0),
    GaussianNB(),
    KNeighborsClassifier(),
    SGDClassifier(random_state=0),
]:
    pipe = make_pipeline(StandardScaler(), estimator)
    scores = cross_val_score(pipe, X_train, y_train, scoring="f1_macro")
    print("### ", str(estimator))
    print(f"CV folds: {scores.round(3)}")
    print(f"Mean: {scores.mean().round(3)}")
    print(f"Std: {scores.std().round(3)}")


###  LinearSVC(class_weight='balanced', max_iter=1000000.0, random_state=0)
CV folds: [0.854 0.893 0.873 0.8   0.853]
Mean: 0.855
Std: 0.031
###  SVC(class_weight='balanced')
CV folds: [0.893 0.964 0.873 0.909 0.818]
Mean: 0.891
Std: 0.048
###  RandomForestClassifier(class_weight='balanced', random_state=0)
CV folds: [0.802 0.875 0.854 0.854 0.873]
Mean: 0.852
Std: 0.026
###  LinearSVC(max_iter=1000000.0, random_state=0)
CV folds: [0.854 0.893 0.873 0.8   0.853]
Mean: 0.855
Std: 0.031
###  SVC()
CV folds: [0.893 0.946 0.854 0.927 0.818]
Mean: 0.888
Std: 0.047
###  RandomForestClassifier(random_state=0)
CV folds: [0.838 0.91  0.817 0.873 0.8  ]
Mean: 0.848
Std: 0.04
###  GaussianNB()
CV folds: [0.607 0.588 0.647 0.762 0.56 ]
Mean: 0.633
Std: 0.071
###  KNeighborsClassifier()
CV folds: [0.856 0.892 0.8   0.834 0.762]
Mean: 0.829
Std: 0.045
###  SGDClassifier(random_state=0)
CV folds: [0.892 0.875 0.836 0.872 0.853]
Mean: 0.866
Std: 0.019


### Parameter tuning

In [17]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        LinearSVC(max_iter=1e6),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced", None],
        "linearsvc__C": [1, 10,100],
        "linearsvc__dual": [True, False],
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc = gsearch.best_estimator_


{'linearsvc__C': 10, 'linearsvc__class_weight': None, 'linearsvc__dual': False, 'pssmselector__iterations': 'all', 'pssmselector__uniref_threshold': 'all'}
0.8875744945064966


In [18]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        SVC(),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "svc__class_weight": ["balanced", None],
        "svc__C": [1, 10,100],
        "svc__gamma": ["scale", 0.1, 0.01, 0.001],
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc = gsearch.best_estimator_


{'pssmselector__iterations': 3, 'pssmselector__uniref_threshold': 'all', 'svc__C': 10, 'svc__class_weight': 'balanced', 'svc__gamma': 'scale'}
0.9095804474036162


### Dimensionality reduction

In [19]:
pca = PCA()
pca.fit(X_train)
csum = np.cumsum(pca.explained_variance_ratio_)
print("Number of components to explain 97% of variance:", np.argmax(csum >= 0.97) + 1)

Number of components to explain 97% of variance: 86


In [20]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        LinearSVC(max_iter=1e6),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced", None],
        "linearsvc__C": [1, 10, 100],
        "linearsvc__dual": [True, False],
        "pca__n_components": np.linspace(0.8, 0.99, 20)
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_lsvc_pca = gsearch.best_estimator_

{'linearsvc__C': 100, 'linearsvc__class_weight': 'balanced', 'linearsvc__dual': False, 'pca__n_components': 0.98, 'pssmselector__iterations': 'all', 'pssmselector__uniref_threshold': 'all'}
0.8872483261409843


In [21]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        SVC(),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "svc__class_weight": ["balanced", None],
        "svc__C": [1, 10, 100],
        "svc__gamma": ["scale", 1, 0.1, 0.01],
        "pca__n_components": np.linspace(0.8, 0.99, 20)
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc_pca = gsearch.best_estimator_

{'pca__n_components': 0.97, 'pssmselector__iterations': 'all', 'pssmselector__uniref_threshold': 'all', 'svc__C': 10, 'svc__class_weight': 'balanced', 'svc__gamma': 'scale'}
0.9347321335863477


### Validation

In [22]:
best_estimator = best_estimator_svc_pca
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.935+-0.032


,Amino,Sugar,Macro,Weighted
precision,0.935,0.897,0.916,0.915
recall,0.879,0.946,0.912,0.914
f1-score,0.906,0.921,0.914,0.914
support,33.000,37.000,70.000,70.000


,Amino,Sugar
Amino,29,4
Sugar,2,35


In [23]:
best_estimator = best_estimator_lsvc_pca
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.887+-0.03


,Amino,Sugar,Macro,Weighted
precision,0.935,0.897,0.916,0.915
recall,0.879,0.946,0.912,0.914
f1-score,0.906,0.921,0.914,0.914
support,33.000,37.000,70.000,70.000


,Amino,Sugar
Amino,29,4
Sugar,2,35
